# Reducción de Ruido con Non-Local Means (NLM)
Probando el filtro **Non-Local Means (NLM)** al decodificar la imagen.

Se aplica en la etapa de **decodificación**.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display, Image
import sys
sys.path.append('../src')
from visualization import show_results_nlm

## Método: Non-Local Means (NLM)
El NLM funciona buscando zonas parecidas en la imagen y promediándolas. Así quita el ruido sin borrar los bordes ni los detalles importantes.

Configuración:
- `-h` o `--h`: **Intensidad (fuerza)**. Determina cuánto ruido se elimina. Valores recomendados: 5 a 30.
- `-t` o `--template_window_size`: **Tamaño del parche**. Es el tamaño del bloque que se usa para comparar similitudes (debe ser impar, por defecto 7).
- `-s` o `--search_window_size`: **Radio de búsqueda**. Es la zona donde se buscan parches similares (debe ser impar, por defecto 21).

--- 
## Caso 1: Imagen `ruido.jpg`

In [ ]:
# 1. Codificar imagen original
!python ../src/NLM.py encode -o ../img/ruido.jpg -e ../img/ruido_encoded

# 2. Decodificar aplicando NLM (h=5, 15, 30)
!python ../src/NLM.py decode -e ../img/ruido_encoded -d ../img/nlm_h5.png --h 5
!python ../src/NLM.py decode -e ../img/ruido_encoded -d ../img/nlm_h15.png --h 15
!python ../src/NLM.py decode -e ../img/ruido_encoded -d ../img/nlm_h30.png --h 30

In [ ]:
paths_ruido = ['../img/nlm_h5.png', '../img/nlm_h15.png', '../img/nlm_h30.png']
show_results_nlm('ruido.jpg', '../img/ruido.jpg', paths_ruido)

--- 
## Caso 2: Imagen `hombre.jpg`

In [ ]:
# 1. Codificar imagen original
!python ../src/NLM.py encode -o ../img/hombre.jpg -e ../img/hombre_encoded

# 2. Decodificar aplicando NLM (h=5, 15, 30)
!python ../src/NLM.py decode -e ../img/hombre_encoded -d ../img/hombre_nlm_h5.png --h 5
!python ../src/NLM.py decode -e ../img/hombre_encoded -d ../img/hombre_nlm_h15.png --h 15
!python ../src/NLM.py decode -e ../img/hombre_encoded -d ../img/hombre_nlm_h30.png --h 30

In [ ]:
paths_hombre = ['../img/hombre_nlm_h5.png', '../img/hombre_nlm_h15.png', '../img/hombre_nlm_h30.png']
show_results_nlm('hombre.jpg', '../img/hombre.jpg', paths_hombre)